In [1]:
# cell-1  
#load and clean the data (removing diacritics and unwanted text)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'iSarcasmEval_sarcasm.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')



# ds = load_dataset('hard')

df = pd.read_csv('iSarcasmEval/train/train.Ar.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3
# display(df.columns)
df_test = pd.read_csv('iSarcasmEval/test/task_A_Ar_test.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3

# df_test = pd.concat([df2, df3], axis=0, ignore_index=True)

# ['id', 'text', 'sarcastic', 'rephrase', 'dialect']
# ['text', 'dialect', 'sarcastic']

      
display(df.columns)
display(df_test.columns)
display(df[:4])

# return 



classes = set(df['sarcastic'].values)
display(classes)

df['sarcastic'] = df['sarcastic'].astype('category')
df['label'] = df['sarcastic'].cat.codes


df_test['sarcastic'] = df_test['sarcastic'].astype('category')
df_test['label'] = df_test['sarcastic'].cat.codes

df = df[['text', 'label']]
df_test = df_test[['text', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))
display(len(df_test))


ds_t = Dataset.from_pandas(df)
ds_v = Dataset.from_pandas(df_test)

# ds = ds['train']
# ds = ds.train_test_split(test_size=0.2)
display(ds_t)
display(ds_v)

max_sequence_length = 128

# classes_num = 6
# display(classes_num)
# display(ds)


models = ['faisalq/bert-base-arabic-wordpiece', 'faisalq/bert-base-arabic-senpiece',
          'faisalq/bert-base-arabic-bbpe']


for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds_t
        dataset_validation = ds_v                                                     
        
      

        def preprocess_function(examples):
            return tokenizer(examples['text'], truncation=True, padding="max_length",
                            max_length=max_sequence_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 10
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 256
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 5, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 5
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('iSarcasmEval_sarcasm_results.csv')
display(best_results)



2024-02-21 06:20:03.781217: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-21 06:20:03.804194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 06:20:04.148771: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Index(['id', 'text', 'sarcastic', 'rephrase', 'dialect'], dtype='object')

Index(['text', 'dialect', 'sarcastic'], dtype='object')

,id,text,sarcastic,rephrase,dialect
0,1,ضبط شخص بدبلوم انتحل صفة طبيب بلد مافيش حد فيها بيشتغل بشهادته,1,شخص ينتحل صفة طبيب ويفتتح عيادة فى بلد فاشلة ضاعت فيها الرقابة,nile
1,2,مش معنى انك قولتلى رايك يبقى أنا هعمل بيه طب افرض انت حمار استحمر أنا كما زيك,1,مش لازم دائما اعمل برأيك,nile
2,3,اية المهلبية دي يصحبي,1,ما هذا الجمال,nile
3,4,الحديث قياس فيه الفضة و فيه النحاس,1,لسانك ترجمان قلبك,magreb


{0, 1}

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-bbpe and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3102 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
5,0.487900,0.831433,0.469286,0.437863
10,0.297700,0.724028,0.650714,0.549733
15,0.216900,1.321860,0.478571,0.446701
20,0.225800,0.738335,0.640000,0.555640
25,0.197300,0.968653,0.549286,0.498631
30,0.164500,0.717224,0.662143,0.571107
35,0.126400,1.126020,0.552857,0.500309
40,0.110200,1.431154,0.505000,0.466299
45,0.077600,1.334997,0.547143,0.493398
50,0.063400,1.425530,0.563571,0.507870


,Model,Accuracy,F1
0,faisalq/bert-base-arabic-bbpe,0.662143,0.571107
3,faisalq/bert-base-arabic-senpiece,0.741429,0.630121
6,faisalq/bert-base-arabic-wordpiece,0.726429,0.618356
